### Reading of Data

In [1]:
import pandas as pd
file_path = r"C:\Users\noco0\Downloads\Cleaning_GenerativeAI_Health_Customer_Insights.csv"
data = pd.read_csv(file_path)
data.fillna('unknown', inplace = True)
data.drop_duplicates(inplace = True)
data.head()

C:\Users\noco0\AppData\Local\Temp\ipykernel_3444\3033018824.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'unknown' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  data.fillna('unknown', inplace = True)


,CustomerID,Age,Gender,Region,HealthGoal,PreferredInteractionChannel,AIInsight,SatisfactionScore,MonthlySpending,EngagementLevel,...,TechSavviness,BMI,ChronicConditions,StressLevel,LastPurchaseDate,ReferralStatus,FeedbackComments,AIRecommendedProducts,AIChatFrequency,PersonalizationScore
0,CUST0001,57.0,Non-Binary,Africa,Fitness,Email,Highly motivated by gamification,7,80.2,High,...,Beginner,27.3,Hypertension,Medium,2023-06-24,No,Not satisfied,Meal Plans,Never,2
1,CUST0002,36.0,Non-Binary,South America,Mental Health,In-person,Highly motivated by gamification,5,498.25,High,...,Advanced,26.5,Multiple,Medium,2020-10-23,Yes,Excellent service,Meal Plans,Daily,2
2,CUST0003,69.0,Male,unknown,Fitness,In-person,Prefers video tutorials,4,429.6,Low,...,Intermediate,24.4,Diabetes,High,2021-02-09,Yes,Average experience,Supplements,Daily,5
3,CUST0004,35.0,Male,North America,Fitness,Phone,Prefers video tutorials,10,187.79,High,...,Intermediate,39.8,unknown,Low,2021-05-03,Yes,Average experience,Fitness Tracker,Rarely,10
4,CUST0005,52.0,Male,Europe,Chronic Disease Management,In-person,Needs regular reminders,9,428.38,Low,...,Intermediate,19.1,Asthma,High,2021-01-20,No,Not satisfied,Meal Plans,Daily,1


### Convert data to langchain documents

In [3]:
from langchain.schema import Document

documents = []

for index, row in data.iterrows():
    doc = Document(page_content=f"Customer ID: {row['CustomerID']}, Age: {row['Age']}, Gender: {row['Gender']}, "
                     f"Region: {row['Region']}, Health Goal: {row['HealthGoal']}, "
                     f"Preferred Interaction Channel: {row['PreferredInteractionChannel']}, "
                     f"AI Insight: {row['AIInsight']}, Satisfaction Score: {row['SatisfactionScore']}, "
                     f"Monthly Spending: {row['MonthlySpending']}, Engagement Level: {row['EngagementLevel']}, "
                     f"Tech Savviness: {row['TechSavviness']}, BMI: {row['BMI']}, "
                     f"Chronic Conditions: {row['ChronicConditions']}, Stress Level: {row['StressLevel']}, "
                     f"Last Purchase Date: {row['LastPurchaseDate']}, Referral Status: {row['ReferralStatus']}, "
                     f"Feedback Comments: {row['FeedbackComments']}, AI Recommended Products: {row['AIRecommendedProducts']}, "
                     f"AI Chat Frequency: {row['AIChatFrequency']}, Personalization Score: {row['PersonalizationScore']}",
        metadata={"CustomerID": row['CustomerID'], "Index": index}
    )
    documents.append(doc)

for doc in documents:
    print(doc)

page_content='Customer ID: CUST0001, Age: 57.0, Gender: Non-Binary, Region: Africa, Health Goal: Fitness, Preferred Interaction Channel: Email, AI Insight: Highly motivated by gamification, Satisfaction Score: 7, Monthly Spending: 80.2, Engagement Level: High, Tech Savviness: Beginner, BMI: 27.3, Chronic Conditions: Hypertension, Stress Level: Medium, Last Purchase Date: 2023-06-24, Referral Status: No, Feedback Comments: Not satisfied, AI Recommended Products: Meal Plans, AI Chat Frequency: Never, Personalization Score: 2' metadata={'CustomerID': 'CUST0001', 'Index': 0}
page_content='Customer ID: CUST0002, Age: 36.0, Gender: Non-Binary, Region: South America, Health Goal: Mental Health, Preferred Interaction Channel: In-person, AI Insight: Highly motivated by gamification, Satisfaction Score: 5, Monthly Spending: 498.25, Engagement Level: High, Tech Savviness: Advanced, BMI: 26.5, Chronic Conditions: Multiple, Stress Level: Medium, Last Purchase Date: 2020-10-23, Referral Status: Yes,

### Split text into chunks

In [4]:
# Convert the DataFrame to a single text string
#####  text_data = "\n".join(data.astype(str).agg(', '.join, axis=1))

text_data = "\n".join(
    data.apply(
        lambda row: (f"Customer {row['CustomerID']} is a {row['Age']}-year-old {row['Gender']} from {row['Region']} "
                     f"with a health goal of {row['HealthGoal']}. They prefer {row['PreferredInteractionChannel']} communication. "
                     f"Their satisfaction score is {row['SatisfactionScore']}, and they spend ${row['MonthlySpending']} monthly. "
                     f"AI insight: {row['AIInsight']}. Feedback: {row['FeedbackComments']}. Stress level is {row['StressLevel']}."),
        axis=1
    )
)


# Initialize the RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50
)

# Split the text data into chunks
chunks = splitter.split_text(text_data)

for i, chunk in enumerate(chunks):
    print(f"Chunk {i + 1}:\n{chunk}\n")

# Step 2: Create Document objects for each chunk
documents = [Document(page_content=chunk) for chunk in chunks]

Chunk 1:
Customer CUST0001 is a 57.0-year-old Non-Binary from Africa with a health goal of Fitness. They prefer Email communication. Their satisfaction score is 7, and they spend $80.2 monthly. AI insight: Highly motivated by gamification. Feedback: Not satisfied. Stress level is Medium.

Chunk 2:
Customer CUST0002 is a 36.0-year-old Non-Binary from South America with a health goal of Mental Health. They prefer In-person communication. Their satisfaction score is 5, and they spend $498.25 monthly. AI insight: Highly motivated by gamification. Feedback: Excellent service. Stress level is Medium.

Chunk 3:
Customer CUST0003 is a 69.0-year-old Male from unknown with a health goal of Fitness. They prefer In-person communication. Their satisfaction score is 4, and they spend $429.6 monthly. AI insight: Prefers video tutorials. Feedback: Average experience. Stress level is High.

Chunk 4:
Customer CUST0004 is a 35.0-year-old Male from North America with a health goal of Fitness. They prefer 

In [5]:
print(documents)

[Document(metadata={}, page_content='Customer CUST0001 is a 57.0-year-old Non-Binary from Africa with a health goal of Fitness. They prefer Email communication. Their satisfaction score is 7, and they spend $80.2 monthly. AI insight: Highly motivated by gamification. Feedback: Not satisfied. Stress level is Medium.'), Document(metadata={}, page_content='Customer CUST0002 is a 36.0-year-old Non-Binary from South America with a health goal of Mental Health. They prefer In-person communication. Their satisfaction score is 5, and they spend $498.25 monthly. AI insight: Highly motivated by gamification. Feedback: Excellent service. Stress level is Medium.'), Document(metadata={}, page_content='Customer CUST0003 is a 69.0-year-old Male from unknown with a health goal of Fitness. They prefer In-person communication. Their satisfaction score is 4, and they spend $429.6 monthly. AI insight: Prefers video tutorials. Feedback: Average experience. Stress level is High.'), Document(metadata={}, pag

### Embedding of documents, Vectorstore and Streamlit

In [1]:
%%writefile app.py
import streamlit as st
from langchain_openai import OpenAI
from langchain_openai import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.docstore.document import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv
import os
import pandas as pd


load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

file_path = r"C:\Users\noco0\Downloads\Cleaning_GenerativeAI_Health_Customer_Insights.csv"
data = pd.read_csv(file_path)
data.fillna('unknown', inplace = True)
data.drop_duplicates(inplace = True)

# Convert the DataFrame to a single text string
#####  text_data = "\n".join(data.astype(str).agg(', '.join, axis=1))

text_data = "\n".join(
    data.apply(
        lambda row: (f"Customer {row['CustomerID']} is a {row['Age']}-year-old {row['Gender']} from {row['Region']} "
                     f"with a health goal of {row['HealthGoal']}. They prefer {row['PreferredInteractionChannel']} communication. "
                     f"Their satisfaction score is {row['SatisfactionScore']}, and they spend ${row['MonthlySpending']} monthly. "
                     f"AI insight: {row['AIInsight']}. Feedback: {row['FeedbackComments']}. Stress level is {row['StressLevel']}."),
        axis=1
    )
)

# Initialize the RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50
)

# Split the text data into chunks
chunks = splitter.split_text(text_data)

for i, chunk in enumerate(chunks):
    print(f"Chunk {i + 1}:\n{chunk}\n")

# Create Document objects for each chunk
documents = [Document(page_content=chunk) for chunk in chunks]

# Set up the FAISS vectorstore from your documents
embedding = OpenAIEmbeddings(api_key=api_key)  # Using OpenAI embeddings
vectorstore = FAISS.from_documents(documents, embedding)

# Streamlit UI Setup
st.title('Health Support Chatbot')
st.write('Ask me health-related questions!')

# Sidebar instructions
st.sidebar.title('Instructions')
st.sidebar.info("This chatbot is designed to provide health support and answer queries.")

# Input from user
query = st.text_input('Enter your query:', '')

if query:
    # Perform the similarity search on the vectorstore
    results = vectorstore.similarity_search(query, k=5)

    # Display similarity search results (showing the top results)
    st.write("**Relevant customer data retrieved from our system:**")
    for result in results:
        st.write(result.page_content)

    # Set up the LLM chain with a prompt template
    prompt = ChatPromptTemplate.from_messages([
        ('system', 'You are a health assistant. Based on the customer data provided, answer the question clearly.'),
        ('human', '{question}'),
    ])

    # Initialize the ChatOpenAI model
    llm = ChatOpenAI(api_key=api_key, temperature=0.3)

    # Combine the relevant data with the user query
    relevant_texts = "\n".join([result.page_content for result in results])

    # Format the question with context
    question_with_context = f"Here is some relevant customer data:\n{relevant_texts}\n\nQuestion: {query}"

    # Format the prompt and invoke the LLM
    formatted_prompt = prompt.format(question=question_with_context)
    response = llm.invoke(formatted_prompt)  # Pass the formatted prompt to the LLM

    # Display the refined response from the LLM
    st.write("**Refined Response from the Health Assistant:**")
    st.write(response)

Overwriting app.py


In [ ]:
!streamlit run app.py